<a href="https://colab.research.google.com/github/devthumos/Processamento_de_Linguagem_Natural/blob/main/Atividade_Avaliativa_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade Avaliativa 02

## Lendo o dataset

In [249]:
import pandas as pd
import numpy as np
import nltk
import re

nltk.download("stopwords")

pd.set_option('max_colwidth', 1000)

path_file = "https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/corpus_tweets/twitter-2013train-A.txt"
dataset = pd.read_csv(path_file, sep="\t", encoding="utf_8", names=["ID", "Polaridade", "Tweet"])
dataset.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ID,Polaridade,Tweet
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out)
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney


## Substituindo algumas coisas

- Substituir :), :(, :/, :3
  - o :3, tem que tomar cuidado pq tem horário assim 5:30 por exemplo, ele tem que estar circundado por palavras ou letras [FEITO]
  - o :/, tem que tomar cuidado pq tem links com http:// por exemplo [FEITO]

- Devemos substituir os links https e etc por link, se possível [FEITO]
  - Tem que terminar com \s ou aspas duplas ou simples
- Devemos substituir os RT's [FEITO]
  - Os RT's sempres são seguidos de um espaço, um @ e um nome de usuário
  - RT\s@[^\s|:]+, podemos simplificar sendo RT algo_que_não_é_espaço_ou__:
- Devemos tirar o &amp [FEITO]
- Devemos substituir o `&lt;`,` &gt`; to nothing [FEITO]
- Devemos substituir o <[3]+ to love you [FEITO]

In [250]:
## Nós podemos depois, substituir os usuários pelo nome "User", para fazer mais sentido para as outras etapas
def simple(sentence):
  pattern1 = r"RT\s(@[^\s|:]+)"  ## To find the RT's
  pattern2 = r"(?:(?:http)|(?:https)|(?:ftp))://[^\s]+"  ## To find links
  pattern3 = r"\\u[0-9]{2,4}"  # Substituir as falhas no encoding
  pattern4 = r"@[^\s|:]+" # Achar os User's
  
  simple = re.sub(pattern1, r"\1", sentence, flags=re.I)
  simple = re.sub(pattern2, "link", simple, flags=re.I)
  simple = re.sub(pattern4, "user", simple, flags=re.I)

  simple = re.sub("&amp", "&", simple, flags=re.I)
  simple = re.sub("&lt;", "", simple, flags=re.I)
  simple = re.sub("&gt;", "", simple, flags=re.I)
  simple = re.sub(pattern3, "", simple, flags=re.I)
  simple = re.sub(":\)", "happy", simple, flags=re.I)
  simple = re.sub("<[3]+", "I love you", simple, flags=re.I)
  simple = re.sub("(?::\()|(?:\s*:/\s*)", " sad ", simple, flags=re.I)
  simple = re.sub("([^0-9]*):3([^0-9]*)", "cutie", simple, flags=re.I)
  return simple

## TESTE
# pattern1 = r"RT\s(@[^\s|:]+)"  ## To find the RT's
# pattern2 = r"((?:(?:http)|(?:https)|(?:ftp))://[^\s|\"']+)"  ## To find links
# print(re.search(pattern1, "RT @AllThingsPLL: Lucy Hale is trending worldwide! The girls from #PrettyLittleLiars are getting some love from the fans tonight. Congra ...", flags=re.I))
# print(re.sub(pattern1, r"\1", "RT @AllThingsPLL: Lucy Hale is trending worldwide! The girls from #PrettyLittleLiars are getting some love from the fans tonight. Congra ...", flags=re.I))
# re.search(pattern2, "Check this video out -- Obama, Sarkozy, Cameron, how many children did you kill this night? http://t.co/AvYCIQJ via @youtube", flags=re.I)
# pattern3 = r"(?::\()|(?:\s*:/\s*)"
# print(re.sub(pattern3, " sad ", "@CherylCole can\u2019t wait for your tour so exited :( !!!! Il see u on Monday i hope :/ remember me Camilla Parker Bowles lol love you so much! Xxxx", flags=re.I))

# text = "Check this video out -- Obama, Sarkozy, Cameron, how many children did you kill this night? http://t.co/AvYCIQJ via @youtube :(. But this is true :)"
# text1 = "Who\u2019s appearing at the Houston Livestock Show and Rodeo come March? That is\u002c other than George Strait?... http://t.co/5u61o6o9"
# pattern2 = r"(?:(?:http)|(?:https)|(?:ftp))://[^\s]+"  ## To find links
# re.search(pattern2, text1, flags=re.I)
# simple(text1)
# re.sub(r"(?::\()|(?:[(?:^http)|(?:^https)|(?:^ftp)]*:/[(?:^http)|(?:^https)|(?:^ftp)]*)", "sad", text1, flags=re.I)

# pattern1 = r"([^0-9]*):3([^0-9]*)"
# text = "@SimonTheShrimp I\u2019m at my friends for lunch tomorrow on North road\u002c did you still want to meet up? :3 xx"
# re.sub(pattern1, r"\1cutie\2", text, flags=re.I)

## Tokenização

!$@#

In [251]:
def tokenize(sentence):
  pattern1 = r"[!\$@#,\.'\"’\)\(\s&\*\-\?~\{\}]"  # Tokenizar, tem que adicionar o + e -, se o mais tiver no final da frase, quer dizer continuação, se tiver em outro lugar, significa more

  sentence = re.split(pattern1, sentence.lower().strip(), flags=re.I)
  sentence = [token for token in sentence if token != ""] ## Para retirar os tokens inválidos ""
  return sentence

# TESTE
# text = "Gas by my house hit $3.39!!!! I\u2019m \"going\" to Chapel Hill on Sat. :)"
# tokenize(simple(text))

In [252]:
dataset["Simplified_Tweet"] = dataset.Tweet.apply(simple)
dataset["Tokenized_Tweet"] = dataset.Simplified_Tweet.apply(tokenize)

dataset.head()

,ID,Polaridade,Tweet,Simplified_Tweet,Tokenized_Tweet
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),Gas by my house hit $3.39!!!! Im going to Chapel Hill on Sat. happy,"[gas, by, my, house, hit, 3, 39, im, going, to, chapel, hill, on, sat, happy]"
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,Theo Walcott is still shitc watch Rafa and Johnny deal with him on Saturday.,"[theo, walcott, is, still, shitc, watch, rafa, and, johnny, deal, with, him, on, saturday]"
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,its not that Im a GSP fanc i just hate Nick Diaz. cant wait for february.,"[its, not, that, im, a, gsp, fanc, i, just, hate, nick, diaz, cant, wait, for, february]"
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),Iranian general says Israels Iron Dome cant deal with their missiles (keep talking like that and we may end up finding out),"[iranian, general, says, israels, iron, dome, cant, deal, with, their, missiles, keep, talking, like, that, and, we, may, end, up, finding, out]"
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,Tehranc Mon Amour: Obama Tried to Establish Ties with the Mullahs link via user No Barack Obama - Vote Mitt Romney,"[tehranc, mon, amour:, obama, tried, to, establish, ties, with, the, mullahs, link, via, user, no, barack, obama, vote, mitt, romney]"


## Retirando Stop Words

In [253]:
def remove_stop_words(token_list):
  stop_list_nltk = nltk.corpus.stopwords.words("english")
  novel_tokens = [no_stop_word for no_stop_word in token_list if no_stop_word not in stop_list_nltk]
  return novel_tokens


## TESTE
# remove_stop_words(dataset["Tokenized_Tweet"][0])

In [254]:
dataset["Tokens_Whithout_Stop_Words"] = dataset["Tokenized_Tweet"].apply(remove_stop_words)
dataset.head()

,ID,Polaridade,Tweet,Simplified_Tweet,Tokenized_Tweet,Tokens_Whithout_Stop_Words
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),Gas by my house hit $3.39!!!! Im going to Chapel Hill on Sat. happy,"[gas, by, my, house, hit, 3, 39, im, going, to, chapel, hill, on, sat, happy]","[gas, house, hit, 3, 39, im, going, chapel, hill, sat, happy]"
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,Theo Walcott is still shitc watch Rafa and Johnny deal with him on Saturday.,"[theo, walcott, is, still, shitc, watch, rafa, and, johnny, deal, with, him, on, saturday]","[theo, walcott, still, shitc, watch, rafa, johnny, deal, saturday]"
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,its not that Im a GSP fanc i just hate Nick Diaz. cant wait for february.,"[its, not, that, im, a, gsp, fanc, i, just, hate, nick, diaz, cant, wait, for, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, february]"
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),Iranian general says Israels Iron Dome cant deal with their missiles (keep talking like that and we may end up finding out),"[iranian, general, says, israels, iron, dome, cant, deal, with, their, missiles, keep, talking, like, that, and, we, may, end, up, finding, out]","[iranian, general, says, israels, iron, dome, cant, deal, missiles, keep, talking, like, may, end, finding]"
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,Tehranc Mon Amour: Obama Tried to Establish Ties with the Mullahs link via user No Barack Obama - Vote Mitt Romney,"[tehranc, mon, amour:, obama, tried, to, establish, ties, with, the, mullahs, link, via, user, no, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tried, establish, ties, mullahs, link, via, user, barack, obama, vote, mitt, romney]"


In [255]:
def stemming(tokens_whithout_stop_words):
  stemmer = nltk.stem.porter.PorterStemmer()
  return [stemmer.stem(token) for token in tokens_whithout_stop_words]

In [256]:
dataset["Stemmed_Tokens"] = dataset["Tokens_Whithout_Stop_Words"].apply(stemming)
dataset.head()

,ID,Polaridade,Tweet,Simplified_Tweet,Tokenized_Tweet,Tokens_Whithout_Stop_Words,Stemmed_Tokens
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),Gas by my house hit $3.39!!!! Im going to Chapel Hill on Sat. happy,"[gas, by, my, house, hit, 3, 39, im, going, to, chapel, hill, on, sat, happy]","[gas, house, hit, 3, 39, im, going, chapel, hill, sat, happy]","[ga, hous, hit, 3, 39, im, go, chapel, hill, sat, happi]"
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,Theo Walcott is still shitc watch Rafa and Johnny deal with him on Saturday.,"[theo, walcott, is, still, shitc, watch, rafa, and, johnny, deal, with, him, on, saturday]","[theo, walcott, still, shitc, watch, rafa, johnny, deal, saturday]","[theo, walcott, still, shitc, watch, rafa, johnni, deal, saturday]"
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,its not that Im a GSP fanc i just hate Nick Diaz. cant wait for february.,"[its, not, that, im, a, gsp, fanc, i, just, hate, nick, diaz, cant, wait, for, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, februari]"
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),Iranian general says Israels Iron Dome cant deal with their missiles (keep talking like that and we may end up finding out),"[iranian, general, says, israels, iron, dome, cant, deal, with, their, missiles, keep, talking, like, that, and, we, may, end, up, finding, out]","[iranian, general, says, israels, iron, dome, cant, deal, missiles, keep, talking, like, may, end, finding]","[iranian, gener, say, israel, iron, dome, cant, deal, missil, keep, talk, like, may, end, find]"
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,Tehranc Mon Amour: Obama Tried to Establish Ties with the Mullahs link via user No Barack Obama - Vote Mitt Romney,"[tehranc, mon, amour:, obama, tried, to, establish, ties, with, the, mullahs, link, via, user, no, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tried, establish, ties, mullahs, link, via, user, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tri, establish, tie, mullah, link, via, user, barack, obama, vote, mitt, romney]"


## Pegando o vocabulário de palavras do corpus de tweets

In [257]:
stemmed_vocabulary_list = []
def stemmed_vocabulary(stemmed_tokens):
  global stemmed_vocabulary_list
  stemmed_vocabulary_list.extend(stemmed_tokens)
  return stemmed_tokens

vocabulary_list = []
def vocabulary(tokens):
  global vocabulary_list
  vocabulary_list.extend(tokens)
  return tokens

In [258]:
dataset.Tokenized_Tweet.apply(vocabulary)
dataset.Stemmed_Tokens.apply(stemmed_vocabulary)

0                                                             [ga, hous, hit, 3, 39, im, go, chapel, hill, sat, happi]
1                                                   [theo, walcott, still, shitc, watch, rafa, johnni, deal, saturday]
2                                                              [im, gsp, fanc, hate, nick, diaz, cant, wait, februari]
3                      [iranian, gener, say, israel, iron, dome, cant, deal, missil, keep, talk, like, may, end, find]
4       [tehranc, mon, amour:, obama, tri, establish, tie, mullah, link, via, user, barack, obama, vote, mitt, romney]
                                                             ...                                                      
9679                                         [user, monday, monday, night, footbal, mind, rt, love, footbal, mnfootng]
9680                                    [know, road, lomardi, start, tonight, set, record, pre, season, mnf, guarante]
9681                                            

## Vocabulário do Corpus

In [259]:
print(np.unique(np.array(vocabulary_list)).tolist())

['%', '%:', '+', '+++', '+0c2', '+1', '+1500', '+1c', '+23', '+2c', '+2c5', '+6c9', '+8', '+plus', '+video', '/', '//', '//played', '/:', '/\\', '/does/', '/via', '0', '00', '000', '000758', '000th', '007', '00:11:11', '00:52cutie9', '00pm', '01', '0118704263', '01943', '01\\', '01c', '02', '02cutie0pm', '03', '039;', '039;s', '04', '04%', '04:', '04\\', '05', '06', '06:', '07', '07%c', '07:', '07:48', '07\\', '07cutie0pm', '08', '08cutie0', '09', '0900', '09109839513', '0930', '0:00', '0:54', '0c', '0ct', '1', '1%', '1+1', '1/2', '1/3rd', '10', '10%', '10+', '10/05', '10/10', '10/10/2011', '10/11', '10/16', '10/17', '10/18', '10/20', '10/21', '10/22', '10/23', '10/24', '10/27', '10/30', '10/6', '10/9', '10/adv', '100', '100%', '100+', '100+s', '1000', '100000000', '10037', '1007', '100c000', '100th', '101', '1039', '1040', '1041', '1043', '105', '106', '1080i', '10:', '10:00', '10:00pm', '10:28pm', '10:29c', '10:46am', '10\\', '10am', '10amc', '10c', '10c000', '10ctwenty', '10cutie0',

## Vocabulário do Corpus com Stemização

In [260]:
print(np.unique(np.array(stemmed_vocabulary_list)).tolist())

['%', '%:', '+', '+++', '+0c2', '+1', '+1500', '+1c', '+23', '+2c', '+2c5', '+6c9', '+8', '+plu', '+video', '/', '//', '//play', '/:', '/\\', '/does/', '/via', '0', '00', '000', '000758', '000th', '007', '00:11:11', '00:52cutie9', '00pm', '01', '0118704263', '01943', '01\\', '01c', '02', '02cutie0pm', '03', '039;', '04', '04%', '04:', '04\\', '05', '06', '06:', '07', '07%c', '07:', '07:48', '07\\', '07cutie0pm', '08', '08cutie0', '09', '0900', '09109839513', '0930', '0:00', '0:54', '0c', '0ct', '1', '1%', '1+1', '1/2', '1/3rd', '10', '10%', '10+', '10/05', '10/10', '10/10/2011', '10/11', '10/16', '10/17', '10/18', '10/20', '10/21', '10/22', '10/23', '10/24', '10/27', '10/30', '10/6', '10/9', '10/adv', '100', '100%', '100+', '1000', '100000000', '10037', '1007', '100c000', '100th', '101', '1039', '1040', '1041', '1043', '105', '106', '1080i', '10:', '10:00', '10:00pm', '10:28pm', '10:29c', '10:46am', '10\\', '10am', '10amc', '10c', '10c000', '10ctwenti', '10cutie0', '10cutie0am', '10cut

## Exemplos de tweets

In [261]:
dataset.head()

,ID,Polaridade,Tweet,Simplified_Tweet,Tokenized_Tweet,Tokens_Whithout_Stop_Words,Stemmed_Tokens
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),Gas by my house hit $3.39!!!! Im going to Chapel Hill on Sat. happy,"[gas, by, my, house, hit, 3, 39, im, going, to, chapel, hill, on, sat, happy]","[gas, house, hit, 3, 39, im, going, chapel, hill, sat, happy]","[ga, hous, hit, 3, 39, im, go, chapel, hill, sat, happi]"
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,Theo Walcott is still shitc watch Rafa and Johnny deal with him on Saturday.,"[theo, walcott, is, still, shitc, watch, rafa, and, johnny, deal, with, him, on, saturday]","[theo, walcott, still, shitc, watch, rafa, johnny, deal, saturday]","[theo, walcott, still, shitc, watch, rafa, johnni, deal, saturday]"
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,its not that Im a GSP fanc i just hate Nick Diaz. cant wait for february.,"[its, not, that, im, a, gsp, fanc, i, just, hate, nick, diaz, cant, wait, for, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, februari]"
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),Iranian general says Israels Iron Dome cant deal with their missiles (keep talking like that and we may end up finding out),"[iranian, general, says, israels, iron, dome, cant, deal, with, their, missiles, keep, talking, like, that, and, we, may, end, up, finding, out]","[iranian, general, says, israels, iron, dome, cant, deal, missiles, keep, talking, like, may, end, finding]","[iranian, gener, say, israel, iron, dome, cant, deal, missil, keep, talk, like, may, end, find]"
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,Tehranc Mon Amour: Obama Tried to Establish Ties with the Mullahs link via user No Barack Obama - Vote Mitt Romney,"[tehranc, mon, amour:, obama, tried, to, establish, ties, with, the, mullahs, link, via, user, no, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tried, establish, ties, mullahs, link, via, user, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tri, establish, tie, mullah, link, via, user, barack, obama, vote, mitt, romney]"


## DataFrame inteiro

In [262]:
dataset

,ID,Polaridade,Tweet,Simplified_Tweet,Tokenized_Tweet,Tokens_Whithout_Stop_Words,Stemmed_Tokens
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),Gas by my house hit $3.39!!!! Im going to Chapel Hill on Sat. happy,"[gas, by, my, house, hit, 3, 39, im, going, to, chapel, hill, on, sat, happy]","[gas, house, hit, 3, 39, im, going, chapel, hill, sat, happy]","[ga, hous, hit, 3, 39, im, go, chapel, hill, sat, happi]"
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,Theo Walcott is still shitc watch Rafa and Johnny deal with him on Saturday.,"[theo, walcott, is, still, shitc, watch, rafa, and, johnny, deal, with, him, on, saturday]","[theo, walcott, still, shitc, watch, rafa, johnny, deal, saturday]","[theo, walcott, still, shitc, watch, rafa, johnni, deal, saturday]"
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,its not that Im a GSP fanc i just hate Nick Diaz. cant wait for february.,"[its, not, that, im, a, gsp, fanc, i, just, hate, nick, diaz, cant, wait, for, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, february]","[im, gsp, fanc, hate, nick, diaz, cant, wait, februari]"
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),Iranian general says Israels Iron Dome cant deal with their missiles (keep talking like that and we may end up finding out),"[iranian, general, says, israels, iron, dome, cant, deal, with, their, missiles, keep, talking, like, that, and, we, may, end, up, finding, out]","[iranian, general, says, israels, iron, dome, cant, deal, missiles, keep, talking, like, may, end, finding]","[iranian, gener, say, israel, iron, dome, cant, deal, missil, keep, talk, like, may, end, find]"
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,Tehranc Mon Amour: Obama Tried to Establish Ties with the Mullahs link via user No Barack Obama - Vote Mitt Romney,"[tehranc, mon, amour:, obama, tried, to, establish, ties, with, the, mullahs, link, via, user, no, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tried, establish, ties, mullahs, link, via, user, barack, obama, vote, mitt, romney]","[tehranc, mon, amour:, obama, tri, establish, tie, mullah, link, via, user, barack, obama, vote, mitt, romney]"
...,...,...,...,...,...,...,...
9679,103158179306807296,positive,RT @MNFootNg It's monday and Monday Night Football is on my mind. RT if you love football! #MNFootNg.,user It's monday and Monday Night Football is on my mind. RT if you love football! #MNFootNg.,"[user, it, s, monday, and, monday, night, football, is, on, my, mind, rt, if, you, love, football, mnfootng]","[user, monday, monday, night, football, mind, rt, love, football, mnfootng]","[user, monday, monday, night, footbal, mind, rt, love, footbal, mnfootng]"
9680,103157324096618497,positive,All I know is the road for that Lomardi start at TONIGHT!!!! We will set a record for a pre-season MNF I can guarantee it.,All I know is the road for that Lomardi start at TONIGHT!!!! We will set a record for a pre-season MNF I can guarantee it.,"[all, i, know, is, the, road, for, that, lomardi, start, at, tonight, we, will, set, a, record, for, a, pre, season, mnf, i, can, guarantee, it]","[know, road, lomardi, start, tonight, set, record, pre, season, mnf, guarantee]","[know, road, lomardi, start, tonight, set, record, pre, season, mnf, guarante]"
9681,100259220338905089,neutral,"All Blue and White fam, we r meeting at Golden Corral for dinner tonight at 6pm....","All Blue and White fam, we r meeting at Golden Corral for dinner tonight at 6pm....","[all, blue, and, white, fam, we, r, meeting, at, golden, corral, for, dinner, tonight, at, 